## Time Series Processing issue
sample notebook to demonstrate issue with initial approach to time series processing 

#### Bring in data for demonstration

In [ ]:
from tickerset import *

In [4]:
#bring in ticker set for analyzing the current processing approach on the ticker time series 
t = TickerSet()

Pulling Ticker data from Yahoo Finance
Creating yf Ticker instance for XLF, fetching history
Creating yf Ticker instance for XHB, fetching history
Creating yf Ticker instance for XLK, fetching history
Creating yf Ticker instance for XLY, fetching history
Creating yf Ticker instance for XLP, fetching history
Creating yf Ticker instance for XRT, fetching history
Creating yf Ticker instance for XLI, fetching history
Creating yf Ticker instance for XLB, fetching history
Creating yf Ticker instance for XTL, fetching history
Creating yf Ticker instance for XLU, fetching history
Creating yf Ticker instance for ^VIX, fetching history


/Users/murphybre/.local/share/virtualenvs/MADS-M1-analysis-of-price-sensitivity-to-v-hcq5-aJA/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


#### Demonstrate Issue "compute price change"

In [5]:
#price change data frame is the result of the "compute price change step "
price_change = t.compute_price_change()

In [17]:
#Add in prev_close as a column as part of the demonstration 
price_change['prev_close'] = price_change['close'].shift(1)

In [39]:
#the columns of interest 
columns = ['ticker','date', 'close', 'open', 'prev_close','price_chg_c2o']

In [41]:
#prev close for first row of XHB is based on last row of XLF 
price_change[price_change.ticker.isin(['XLF', 'XHB'])][columns].iloc[5782:5790]


,ticker,date,close,open,prev_close,price_chg_c2o
5782,XLF,2021-12-14,38.67,38.39,38.45,-0.001560
5783,XLF,2021-12-15,38.78,38.77,38.67,0.002586
5784,XLF,2021-12-16,39.27,39.23,38.78,0.011604
5785,XHB,2006-02-06,38.49,39.17,39.27,-0.002546
5786,XHB,2006-02-07,37.51,38.02,38.49,-0.012211
5787,XHB,2006-02-08,38.14,37.45,37.51,-0.001600
5788,XHB,2006-02-09,37.86,38.06,38.14,-0.002098
5789,XHB,2006-02-10,37.45,37.58,37.86,-0.007396


#### Demonstrate issue with rolling volume 

In [34]:
rolling_volume = t.compute_rolling_volume(price_change)

In [37]:
#volume_rollmed for XHB is computed accross a window including XLF 
columns = ['ticker','date', 'volume', "volume_rollmed","volume_diff_to_med","volume_pchg_from_med"]
rolling_volume.iloc[5782:5790][columns]

,ticker,date,volume,volume_rollmed,volume_diff_to_med,volume_pchg_from_med
5782,XLF,2021-12-14,51734400,47958300.0,3776100.0,0.078737
5783,XLF,2021-12-15,70424900,47958300.0,22466600.0,0.468461
5784,XLF,2021-12-16,71156600,48038950.0,23117650.0,0.481227
5785,XHB,2006-02-06,71000,47958300.0,-47887300.0,-0.998520
5786,XHB,2006-02-07,203500,47958300.0,-47754800.0,-0.995757
5787,XHB,2006-02-08,553500,47958300.0,-47404800.0,-0.988459
5788,XHB,2006-02-09,70700,46871200.0,-46800500.0,-0.998492
5789,XHB,2006-02-10,264100,45754100.0,-45490000.0,-0.994228


In [ ]:
#same idea is true for the rolling volume and bollinger and Z

#To correct for this, we either need to 
# A) calculate the aggregations for each ticker time series separately before joining them together
# B) join them and then compute the aggregations group wise (split apply transform )


#For approach A

#Fetch Ticker, compute time series aggregation
#---- move processing from Ticker Set to Ticker, TickerSet can still union and join with VIX
